In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document

def load_and_chunk_documents(file_path: str) -> List:
    """
    Загружает документ из файла и разбивает его на чанки.
    """
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", " ", ""]
    )
    chunked_docs = text_splitter.split_documents(documents)
    print(f"Документ '{file_path}' разбит на {len(chunked_docs)} чанков.")
    return chunked_docs

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from typing import List
from langchain_core.documents import Document
import os
import shutil

def create_vector_store(chunked_docs: List, collection_name: str = "credit_platform_docs"):
    """
    Создает векторное хранилище из чанков документов.
    """
    persist_dir = "chroma_db"
    if os.path.exists(persist_dir):
        shutil.rmtree(persist_dir)

    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    vector_store = Chroma.from_documents(
        documents=chunked_docs,
        embedding=embeddings,
        collection_name=collection_name,
        persist_directory=persist_dir
    )

    print(f"Векторное хранилище '{collection_name}' создано и содержит {len(vector_store.get()['ids'])} документов.")
    return vector_store

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_together import ChatTogether
from langchain_chroma import Chroma
from rag_components.data_loader import load_and_chunk_documents
from rag_components.vector_store import create_vector_store

# Шаблон промпта, который инструктирует модель
RAG_PROMPT_TEMPLATE = """
Используйте предоставленный ниже контекст, чтобы ответить на вопрос пользователя.
Если вы не знаете ответа на основе предоставленного контекста, просто скажите, что вы не знаете. Не пытайтесь выдумывать ответ.

Контекст:
{context}

Вопрос:
{question}

Ответ:
"""

def format_docs(docs):
    """
    Форматирует список документов в единую строку.
    """
    return "\n\n".join(doc.page_content for doc in docs)

def create_qa_rag_chain(vector_store: Chroma):
    """
    Создает и возвращает RAG-цепочку для ответов на вопросы.
    """
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})

    llm = ChatTogether(
        model="deepseek-ai/DeepSeek-V3",
        temperature=0,
        together_api_key="5eb69efdc6d0b50e93018ea0f02f86710db98fb253ca58dd2541d033a3df3d8a"
    )
    prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

    rag_chain = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )
    return rag_chain

In [ ]:
import os
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from tools.qa_tool import create_qa_rag_chain
from rag_components.data_loader import load_and_chunk_documents
from rag_components.vector_store import create_vector_store


def create_p2p_agent():
    """
    Создает и настраивает агента для P2P платформы.
    """
    # --- Инициализация инструментов ---

    # Инструмент для ответов на вопросы (RAG)
    print("Инициализация RAG-инструмента...")
    chunked_docs = load_and_chunk_documents("agent/data/testdata.md")
    vector_store = create_vector_store(chunked_docs, "p2p_agent_db")
    agent_executor = create_qa_rag_chain(vector_store)
    
    print("Агент успешно создан.")
    return agent_executor


In [ ]:
p2p_agent_executor = create_p2p_agent()

print("\n--- Тест 1 ---")
response1 = p2p_agent_executor.invoke("Какие у вас гарантии безопасности?")
print("Ответ интеллектуального агента:", response1)

print("\n--- Тест 2 ---")
response2 = p2p_agent_executor.invoke("Хочу вложить 3000eth на 3 месяца и на 1000usdt")
print("Ответ интеллектуального агента:", response2)

print("\n--- Тест 3 ---")
response3 = p2p_agent_executor.invoke("Какой минимальный срок для вклада?")
print("Ответ интеллектуального агента:", response3)

Инициализация RAG-инструмента...
Документ 'data/testdata.md' разбит на 3 чанков.


c:\hackaton\singularity\agent\rag_components\vector_store.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\hackaton\singularity\agent\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Векторное хранилище 'p2p_agent_db' создано и содержит 3 документов.
Агент успешно создан.

--- Тест 1. Вопрос о платформе ---
Ответ интеллектуального агента: На основе предоставленного контекста можно выделить следующие гарантии безопасности на платформах взаимного кредитования (P2P):  

1. **Оценка кредитоспособности заемщиков** — платформы проверяют заявки и анализируют риски перед их публикацией.  
2. **Репутация платформы** — рекомендуется выбирать сервисы с прозрачной политикой и проверенной историей работы.  
3. **Страхование вкладов** — некоторые платформы предлагают страховые механизмы или используют фонды гарантирования для защиты инвесторов.  
4. **Технологии (блокчейн и смарт-контракты)** — автоматизация процессов снижает риски мошенничества и повышает прозрачность операций.  

Однако важно помнить, что **кредитные риски** (невозврат займов) и **платформенные риски** (надежность сервиса) остаются, поэтому инвесторам стоит диверсифицировать вложения и тщательно изучать услови